In [1]:
import numpy as np
import pandas as pd
import sys

from tqdm.notebook import tqdm

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer


In [2]:
df = pd.read_pickle("data/similar_sentences.pkl")
print(df.shape)
df.head()


(3028, 4)


,id,sentence1,sentence2,avg
0,ACN2_111,Atorga per primer cop les mencions Encarna San...,Creen la menció M. Encarna Sanahuja a la inclu...,3.50
1,Oscar2_211,"Finalment, afegiu-hi els bolets que haureu sal...","Finalment, poseu-hi les minipastanagues tallad...",1.25
2,ACN2_574,El TC suspèn el pla d'acció exterior i de rela...,El Constitucional manté la suspensió del pla e...,3.67
3,Viqui2_341,Virgin Galactic va ser fundada el 2004 per l'e...,Virgin Atlantic és una de les aerolínies de Ri...,2.25
4,ACN2_1184,Identifiquen un nou biomarcador per realitzar ...,Nous tractaments contra el càncer de mama,2.00


In [3]:
SAMPLE_SIZE = 200
df = df.sample(n=SAMPLE_SIZE, random_state=42)
sentences = (
    pd.concat([df["sentence1"], df["sentence2"]], axis=0)
    .reset_index(drop=True)
    .rename("Sentences")
)
print(sentences.shape)
sentences.head()


(400,)


0    Participà en els conclaves de 1667, 1676, 1689...
1    Educació es compromet a desencallar el project...
2    Exposició per treballar temes de sexualitat, d...
3    Està situada a l'extrem sud-oriental del poble...
4    L'alçada a la qual els podem trobar és de fins...
Name: Sentences, dtype: object

In [ ]:
model_name = "PlanTL-GOB-ES/roberta-base-ca"
# model_name = "multi-qa-MiniLM-L6-cos-v1"
model = SentenceTransformer(model_name)


In [5]:
# 400 sentences --> 10.4s -->
# 400 sentences -->  2.5s --> 0.59 MB

sentence_vecs = model.encode(sentences)
print(f"{sys.getsizeof(sentence_vecs)/float(1<<20):,.2f} MB")


1.17 MB


In [6]:
a = np.array([10, 1, 5, 2, 3, 4])

ind = np.argpartition(a, -5)[-5:]
ind[np.argsort(a[ind])[::-1]]


array([0, 2, 5, 4, 3], dtype=int64)

In [7]:
pairs = []

for i in tqdm(range(len(sentence_vecs))):
    other_sentences = np.delete(sentence_vecs.copy(), i, 0)
    cos_scores = cosine_similarity([sentence_vecs[i]], other_sentences).flatten()
    # most_similar_index = cos_scores.argmax()
    # most_similar_index += 1 if (most_similar_index >= i) else 0

    # Get, sort and adjust the top 5 cosines scores
    top5 = np.argpartition(cos_scores, -5)[-5:]
    top5_sorted = top5[np.argsort(cos_scores[top5])[::-1]]
    top5_adj = [ind + 1 if ind >= i else ind for ind in top5_sorted]

    pairs.append((i, top5_adj, list(cos_scores[top5_sorted])))


  0%|          | 0/400 [00:00<?, ?it/s]

In [8]:
# Export the dataframes

# df.to_pickle("samples/df_sentences.pkl")
# pd.DataFrame(pairs).to_pickle("samples/pairs_sentences.pkl")
# pd.DataFrame(pairs).to_pickle("samples/pairs_sentences_QA.pkl")


In [9]:
# Precision @1 - roberta-base-ca           - 0.9475 - 21
# Precision @1 - multi-qa-MiniLM-L6-cos-v1 - 0.87   - 52
sum([1 for (a, b, _) in pairs if abs(a - b[0]) == (len(sentences) / 2)]) / float(
    len(sentences)
)


0.9475

In [10]:
different = [a for (a, b, _) in pairs if abs(a - b[0]) != (len(sentences) / 2)]
different2 = [a - 200 if a >= (len(sentences) / 2) else a for a in different]
df2 = df.iloc[different2]

print(len(different2))
print(df2["avg"].mean())
print()

# Show the differences with their original index; sorted and grouped
pd.concat(
    [
        pd.DataFrame(different, different2)
        .reset_index()
        .rename({"index": "group"}, axis=1),
        df2.reset_index(),
    ],
    axis=1,
).set_index(0).sort_values(by=["group", 0])


21
2.0442857142857145



,group,index,id,sentence1,sentence2,avg
0,,,,,,
6,6,52,Oscar2_205,"L'edifici té dues plantes, i a la planta baixa...",També hi ha una terrassa en aquesta planta.,2.00
206,6,52,Oscar2_205,"L'edifici té dues plantes, i a la planta baixa...",També hi ha una terrassa en aquesta planta.,2.00
34,34,1731,ACN2_1028,La CUP demana la comissió d'investigació,El Parlament constituirà una comissió d'invest...,1.75
234,34,1731,ACN2_1028,La CUP demana la comissió d'investigació,El Parlament constituirà una comissió d'invest...,1.75
50,50,565,ACN2_1139,"La dona duia, dins d'una bossa de mà, un cisel...",L'arrestat hauria intentat agafar una bossa de...,1.67
250,50,565,ACN2_1139,"La dona duia, dins d'una bossa de mà, un cisel...",L'arrestat hauria intentat agafar una bossa de...,1.67
65,65,2644,ACN2_785,L'agressor va anar a casa a agafar dos ganivet...,"L'atracador, que ja ha ingressat a presó, anav...",1.50
82,82,2874,Oscar2_358,"La presentació anirà a càrrec d'Ignasi Moreta,...","A càrrec de Josep M. Salrach, de la Universita...",2.67
289,89,51,Viqui2_28,També s'ha especulat amb altres possibles expl...,Però també hi ha altres explicacions possibles.,3.67
